In [ ]:
import pandas as pd

dir_path = 'I:/Data_for_practice/NA Sci-Tech Nexus/Data/'

cpc_info = pd.read_csv(dir_path+"uspto_cpc_long_utility.csv")

In [19]:
citations = pd.read_csv(dir_path+"citations_utility_domestic24.csv")
citations.head()

,citing_patent_id,cited_patent_id
0,10000000,5093563
1,10000000,5751830
2,10000001,7804268
3,10000001,9022767
4,10000001,9090016


In [11]:
patents = pd.read_csv(dir_path+"patent_utility_dates_usonly.csv")
patents['filing_year'] = patents['filing_date'].astype(str).str[:4].astype(int)
patents.head()

,patent_id,grant_date,filing_date,usweight,filing_year
0,3930273,1976-01-06,1975-01-10,1.0,1975
1,3930274,1976-01-06,1974-10-11,1.0,1974
2,3930275,1976-01-06,1975-03-31,1.0,1975
3,3930276,1976-01-06,1972-08-07,1.0,1972
4,3930277,1976-01-06,1974-08-21,1.0,1974


In [20]:
citing_patents = patents[['patent_id', 'filing_year']].rename(columns={'patent_id': 'citing_patent_id', 'filing_year': 'citing_fyear'})
citations = citations.merge(citing_patents, on='citing_patent_id', how='inner')
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear
0,10000000,5093563,2015
1,10000000,5751830,2015
2,10000007,1404819,2016
3,10000007,2085447,2016
4,10000007,2241645,2016


In [21]:
cited_patents = patents[['patent_id', 'filing_year']].rename(columns={'patent_id': 'cited_patent_id', 'filing_year': 'cited_fyear'})
citations = citations.merge(cited_patents, on='cited_patent_id', how='inner')
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear
0,10000000,5093563,2015,1989
1,10000000,5751830,2015,1995
2,10000007,4342216,2016,1980
3,10000007,4604890,2016,1985
4,10000007,4689857,2016,1987


In [22]:
citations['fcitelag'] = citations['citing_fyear'] - citations['cited_fyear']
citations = citations[citations['fcitelag'] >= 0]
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fcitelag
0,10000000,5093563,2015,1989,26
1,10000000,5751830,2015,1995,20
2,10000007,4342216,2016,1980,36
3,10000007,4604890,2016,1985,31
4,10000007,4689857,2016,1987,29


In [24]:
cpc_info['cpcagg'] = cpc_info['cpc_subclass'].astype(str).str[0]
us_patents = patents.merge(cpc_info, on='patent_id', how='inner')
us_patents.head()

,patent_id,grant_date,filing_date,usweight,filing_year,cpc_subclass,cpc_group,cpc_subgroup,cpcagg
0,3930273,1976-01-06,1975-01-10,1.0,1975,A61G,A61G7,A61G7/0507,A
1,3930273,1976-01-06,1975-01-10,1.0,1975,A61G,A61G7,A61G7/0509,A
2,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B7,B63B7/085,B
3,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B34,B63B34/00,B
4,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B35,B63B35/34,B


In [ ]:
# Focal patents
focal_patents = us_patents[us_patents['filing_year'] == 2000]
focal_patents.head()

,patent_id,grant_date,filing_date,usweight,filing_year,cpc_subclass,cpc_group,cpc_subgroup,cpcagg
5130337,6076844,2000-06-20,2000-01-07,1.000000,2000,B60R,B60R3,B60R3/00,B
5147039,6083063,2000-07-04,2000-01-13,1.000000,2000,B63H,B63H11,B63H11/01,B
5183071,6095256,2000-08-01,2000-01-19,1.000000,2000,B44B,B44B5,B44B5/0033,B
5185308,6096107,2000-08-01,2000-01-03,0.666667,2000,B24D,B24D3,B24D3/06,B
5185309,6096107,2000-08-01,2000-01-03,0.666667,2000,B24D,B24D11,B24D11/005,B


In [ ]:
# Focal citations
focal_citations = focal_patents.merge(citations, left_on='patent_id', right_on='citing_patent_id', how='inner')
focal_citations = focal_citations[['citing_patent_id', 'cited_patent_id', 'citing_fyear', 'cited_fyear']]
focal_citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear
0,6076844,4102432,2000,1977
1,6076844,5054799,2000,1990
2,6076844,5593167,2000,1994
3,6083063,4373919,2000,1980
4,6083063,4538996,2000,1983


In [ ]:
# Newer citations
newer_citations = citations[(citations['citing_fyear'] > 2000) & (citations['citing_fyear'] < 2006)]
newer_citations = newer_citations.merge(focal_citations['cited_patent_id'], on='cited_patent_id', how='inner')
newer_citations['bit'] = 1
newer_citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fcitelag,bit
0,10000788,4098876,2002,1976,26,1
1,10000788,4098876,2002,1976,26,1
2,10000788,4098876,2002,1976,26,1
3,10000788,4098876,2002,1976,26,1
4,10000788,4098876,2002,1976,26,1


In [33]:
focal_cited_by_newer = citations[(citations['citing_fyear'] > 2000) & (citations['citing_fyear'] < 2006)]
focal_cited_by_newer = focal_cited_by_newer.merge(focal_patents[['patent_id']], left_on='cited_patent_id', right_on='patent_id', how='inner')
focal_cited_by_newer['fit'] = 1
focal_cited_by_newer.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fcitelag,patent_id,fit
0,10000788,6472166,2002,2000,2,6472166,1
1,10000788,6764648,2002,2000,2,6764648,1
2,10000788,6764648,2002,2000,2,6764648,1
3,10000788,6764648,2002,2000,2,6764648,1
4,10000788,6764648,2002,2000,2,6764648,1


In [34]:
data = newer_citations.merge(focal_cited_by_newer, on=['citing_patent_id', 'cited_patent_id'], how='outer').fillna(0)
data['fitbit'] = data['fit'] * data['bit']
data['cdindex'] = -2 * data['fitbit'] + data['fit']
data.head()

MemoryError: Unable to allocate 21.5 GiB for an array with shape (6, 481300899) and data type float64

In [ ]:
# CD Index 특허별 평균 계산
cdindex_summary = data.groupby('focal_patent_id')['cdindex'].mean().reset_index()
cdindex_summary.rename(columns={'cdindex': 'mean_cdindex'}, inplace=True)

# 결과 저장
cdindex_summary.to_csv("c:\\Data_patents\\uspto_updates_patview\\cdindex_results.csv", index=False)